<a href="https://colab.research.google.com/github/eric88525/Tbrain_NLP_game/blob/master/BIN_package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 洗錢新聞分類器

In [ ]:
!nvidia-smi

Mon Aug  3 15:34:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
!pip install transformers
!pip install opencc
!pip install pyprind
!pip install zhon
!pip install -U ckiptagger[tfgpu,gdown]

     |████████████████████████████████| 778kB 5.1MB/s 
     |████████████████████████████████| 1.1MB 29.6MB/s 
     |████████████████████████████████| 890kB 39.5MB/s 
     |████████████████████████████████| 3.0MB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=cba3657127559a23ae6773f89d4896d5d59da41d624b9ae1f9e4676a007ff5df
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.3MB 4.9MB/s 
     |████████████████████████████████| 102kB 4.1MB/s 
  Created wheel for zhon: filename=zhon-1.1.5-cp36-none-any.whl size=84293 sha256=7cca9897ea1d803ca00997d732a9731a730834189dc664f1ca29dc1f496d3fc2
  Stored in directory: /root/.cache/pip/wheels/0e/93/5a/ad2f403c359ba996e33c21bf18611d921413df9740ede2fcf4
Successfully built zhon
     |████████████████████████████████| 411.0MB 41kB/s 
     |█████████████████████████████

In [ ]:
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
from zhon.hanzi import non_stops,stops
import numpy as np
import random
print(torch.cuda.is_available())
!nvidia-smi

True
Mon Aug  3 15:36:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [ ]:
def eval(pred, ans):
    if bool(pred) is not bool(ans):
        return 0
    elif not pred and not ans:
        return 1
    else:
        pred = set(pred)
        ans = set(ans)
        interaction_len = len(pred & ans)
        if interaction_len == 0:
            return 0

        pred_len = len(pred)
        ans_len = len(ans)
        return 2 / (pred_len / interaction_len + ans_len / interaction_len)


def eval_all(pred_list, ans_list):
    assert len(pred_list) == len(ans_list)
    return sum(eval(p, a) for p, a in zip(pred_list, ans_list)) / len(pred_list)

In [ ]:
cd ./drive/My Drive/Colab Notebooks/TBrain/nosep

/content/drive/My Drive/Colab Notebooks/TBrain/nosep


In [ ]:
class bertwwmBIN(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmBIN,self).__init__()
        self.bert_model = BertModel.from_pretrained(model_name)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,1)
            ,nn.Sigmoid()
        )
    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
class bertwwmQA(nn.Module):
    def __init__(self,model_name,config):
        super(bertwwmQA,self).__init__()
        #self.bert_model = BertModel.from_pretrained("hfl/chinese-roberta-wwm-ext-large")
        
        self.bert_model = BertModel.from_pretrained(model_name,config = config)
        self.bi_decoder = nn.Sequential(
            nn.Linear(config.hidden_size,config.hidden_size)
            ,nn.Dropout(0.1)
            ,nn.ReLU()
            ,nn.Linear(config.hidden_size,2)
        )
        #self.start()

    def start(self):
      nn.init.xavier_uniform_(self.bi_decoder[0].weight)
      nn.init.constant_(self.bi_decoder[0].bias, 0)
      nn.init.xavier_uniform_(self.bi_decoder[3].weight)
      nn.init.constant_(self.bi_decoder[3].bias, 0)

    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None):
       # print(f'receive input_ids {input_ids}')

        cls = self.bert_model(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)[1]
        binary = self.bi_decoder(cls)
        return binary

In [ ]:
class QAFilter():
  def __init__(self,model_path): 
    
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmQA(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.c2tw = OpenCC('s2t') # china to tw
    self.tw2c = OpenCC('t2s') # tw to china

  def filter(self,content,name_list):
    # name_list : a list like: ['蔡英文','陳水扁']
    # content  : a new "繁體新聞"
    # 會回傳像是 [1,0,0,1] 代表第0和3的人名是對的
    content = self.tw2c.convert(content)
    pred_nlist = []
    with torch.no_grad():
      for n in name_list:
        nc = self.tw2c.convert(n)
        token_tensor = self.tokenizer.encode_plus(str(nc),str(content),max_length=512,truncation=True,pad_to_max_length=True)
        token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
        segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
        mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
        pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment).argmax(dim=1)
        #print(n,pred.item())
        if pred.item() == 1:
          pred_nlist.append(n)
    return pred_nlist

In [ ]:
class AML_BIN():
  def __init__(self,model_path): 
    from opencc import OpenCC 
    # device
    self.device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
    # model & tokenizer
    model_type = 'hfl/chinese-roberta-wwm-ext'
    config = BertConfig.from_pretrained(model_type,output_hidden_states=True)
    self.tokenizer = BertTokenizer.from_pretrained(model_type)
    self.model = bertwwmBIN(model_type,config).to(self.device)
    self.model.load_state_dict(torch.load(model_path)) 
  
    # 繁簡轉換
    self.tw2c = OpenCC('t2s') # tw to china

  def bin(self,content):
    # content  : a new "繁體新聞"
    # 會回傳 True / False
    content = self.tw2c.convert(content)
    with torch.no_grad():
      token_tensor = self.tokenizer.encode_plus(str(content),max_length=512,truncation=True,pad_to_max_length=True)
      token = torch.tensor(token_tensor['input_ids']).unsqueeze(0).to(self.device)
      segment = torch.tensor( token_tensor['token_type_ids']).unsqueeze(0).to(self.device)
      mask = torch.tensor( token_tensor['attention_mask'] ).unsqueeze(0).to(self.device)
      pred = self.model(input_ids=token,attention_mask=mask,token_type_ids=segment)
      #print(pred.item())
      if pred.item() > 0.5:
        return True
      else:
        return False

In [ ]:
class mydata_nosep():
    def __init__(self,train_path,test_path):

        self.cc = OpenCC('t2s')
        test_data = pd.read_csv(test_path)
        # CONTENT / NAME
        test_x = [self.clean_string(i) for i in test_data['full_content'].values.tolist()]
        test_y = [self.name_list(i) for i in test_data['name'].values.tolist()]
        self.test = []  # CONTENT / NAME         
        for i in range(len(test_x)):
          self.test.append([test_x[i],test_y[i]])
        print(f'Test: {len(self.test)}')

    def name_list(self,name_list_str):
        ls = ast.literal_eval(name_list_str)
        res = []
        #cc = OpenCC('t2s')
        for n in ls:
          res.append(n)
        return res        
    def clean_string(self,content):
      content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
      content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
      content = re.sub(r"[%s]+" %stops, "。",content)
      #content = self.cc.convert(content)
      return content

In [ ]:
def test(data,ws,pos,ner,QA,BIN):
    print('Testing...')
    c_to_tw = OpenCC('s2t') # chi to tw
    tw_to_c = OpenCC('t2s')
    all_predlist = []
    all_labellist = []
    with torch.no_grad():
      for content,nlabel in data.test: # 都是繁中文
        if BIN.bin(content):
          ckip_nlist = []
          pred_nlist = []
          ws_results = ws([content])
          pos_results = pos(ws_results)
          ner_results = ner(ws_results, pos_results)
          for name in ner_results[0]:
            if name[2] == 'PERSON':
              ckip_nlist.append(name[3])
          ckip_nlist = set([x for x in ckip_nlist if len(x)>1])
          pred_nlist = QA.filter(content,ckip_nlist)
          all_labellist.append(nlabel)
          all_predlist.append(pred_nlist)
          #print(f'CKIP:{ckip_nlist} PRED:{pred_nlist} LABEL:{nlabel}')
        else:
          all_labellist.append(nlabel)
          all_predlist.append([])
                      
    return all_predlist,all_labellist

In [ ]:
"""
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")
print('CKIP finish')
"""

CKIP finish


In [ ]:
#qa = QAFilter('./saved_models/qaFilter.pt')
bin = AML_BIN('./saved_models/BIN.pt')

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [ ]:
import csv
data = pd.read_csv('2020-08-03.csv')

with open('./bin_cmp_0803.csv','w',encoding="utf-8",newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['idx','full_content','bin','my_bin'])
  for i in range(len(data)):
    b = int(bin.bin(content))
    if not (data['binary'][i] and b):
      writer.writerow([ i+2 , data['article'][i] , data['binary'][i],b])
            

#content = '生產老字號零食「蝦味先」的裕榮食品公司去年驚爆使用過期原料生產食品，負責人張延恭、廠長李亮名等4人被依詐欺等罪起訴，高雄市衛生局另以裕榮公司違反《食安法》開罰720萬元，裕榮公司不服處分提起行政訴訟，高雄高等行政法院認為，衛生局開罰的事由已進入法院審理中，衛生局在判決定讞前先開罰，違反一事不二罰原則，判衛生局應返還裕榮公司720萬元罰金。可上訴。工廠位於高雄大寮區的裕榮公司去年5月間因爆出使用過期原料產製部分「蝦味先」品項產品，遭高市衛生局與雄檢會同查獲，檢方偵結後認定裕榮公司及88歲負責人張延恭等4人為節省成本使用過期原料，依詐欺取財及違反《食品安全衛生管理法》「製造、販賣逾有效日期食品」等罪嫌起訴，全案現正由高雄地院審理中。裕榮公司提起行政訴訟指出，該公司因使用逾期「無水檸檬酸」柴魚粉製造「蝦味先泡菜」、「蝦味先日式照燒」、「蜜烤魷魚」及使用逾期紅麴粉製造「發芽大豆養生粉─紅麴」等產品，2017年6月間遭高市衛生局以違反食安法規定開罰720萬元，並要求將違規產品回收下架。裕榮公司認為，全案已經檢方起訴由法院審理中，衛生局率先開罰無理由。高雄高等行政法院審理後則認為，針對裕榮公司使用過期原料一事，衛生局開罰的違章事實與高雄地檢署偵查起訴的原因事實相同，依「刑罰優先」原則，衛生局應暫時停止行政秩序罰，並移送檢察官偵查，等到判決結果出爐後再決定是否處以行政罰。本案既然已經檢方起訴由高雄地院審理中，則在刑事判決定讞前，衛生局的裁罰權仍屬暫時停止狀態，因此判決將衛生局原處分撤銷，判衛生局應先返還已對裕榮公司執行的720萬元罰鍰。可再上訴。（王吟芳／高雄報導）'
#bin.bin(content)

In [ ]:
for i in range(1,11):
  trainpath = ''
  testpath = './datasetALL/dataset' + str(i) + '/tbrain_test.csv'
  print(testpath)
  mydata = mydata_nosep('./data3/tbrain_train.csv',testpath)
  pred,label = test(mydata,ws,pos,ner,qa,bin)
  print(pred)
  print(label)
  print(eval_all(pred,label))
  print('=================================================')

./datasetALL/dataset1/tbrain_test.csv
Test: 491
Testing...
[[], [], [], [], [], [], [], [], [], [], [], [], [], ['賴俊吉'], [], [], ['陳揚宗'], [], [], [], [], [], [], ['王益洲'], [], [], [], [], [], [], [], [], [], [], ['李士綸', '吳哲瑋李'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['林志聰', '伍政山'], [], [], [], [], [], [], [], [], [], [], [], [], [], ['張承平'], [], [], [], [], [], [], ['雷俊玲'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['徐仲榮', '陳仕修'], [], ['林煒智', '周漢祥', '林睿耆', '林昱伯'], [], [], ['童仲彥', '秦儷舫'], [], [], [], [], ['蘇怡寧'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['王禹婕', '林育賢'], [], [], [], [], [], [], [], [], [], [], [], ['詹舜淇'], [], [], [], [], [], [], [], [], [], [], [], [], [], ['黃世陽'